In [1]:
import dataloader
import argparse
import importlib
importlib.reload(dataloader)

<module 'dataloader' from '/u/nyw6dh/HCDM/Experiment/LearnabilityLock/dataloader.py'>

In [2]:
# Arguments
parser = argparse.ArgumentParser()
parser.add_argument('--dataset', type=str, default='cora')
parser.add_argument('--gpu_id', type=int, default=0)
parser.add_argument('--config', type=str, default='config.yaml')
args = parser.parse_args("")

In [3]:
# Load configuration
import yaml
from yaml import SafeLoader
config = yaml.load(open('config.yml'), Loader=SafeLoader)[args.dataset]

In [4]:
# Load the data
dataset = dataloader.load_DGL('cora')[0]

  NumNodes: 2708
  NumEdges: 10556
  NumFeats: 1433
  NumClasses: 7
  NumTrainingSamples: 140
  NumValidationSamples: 500
  NumTestSamples: 1000
Done loading data from cached files.


In [8]:
# 3. Attack ===================

# Set up surrogate model
# surrogate = models.GCN(
#   in_size=dataset.ndata['feat'].shape[1],
#   out_size=int(dataset.ndata['label'].max()) + 1,
#   hid_size=config['hid_size'],
#   lr=config['lr'],
#   dropout=config['dropout'],
#   weight_decay=config['weight_decay']
# )

# surrogate(dataset, dataset.ndata['feat'])
surrogate.fit(dataset, dataset.ndata['feat'], dataset.ndata['label'], 100)


GCN Training: 100%|██████████| 100/100 [00:00<00:00, 112.10it/s, loss=0.76]


In [292]:
import dgl
import torch
import models
import importlib
importlib.reload(models)
from tqdm import tqdm

class LearnabilityLock():
  def __init__(self, graph: dgl.DGLGraph):
    self.graph = graph
    
  def set_protected(self, protected: torch.Tensor) -> None:
    assert self.graph.num_nodes() == protected.shape[0]
    assert protected.dtype == torch.bool
    self.graph.ndata['protected'] = protected

  def designate_protected_random(self, node_ct: int) -> None:
    """
    Randomly select ct nodes to be the protected set
    """
    sample = torch.rand(self.graph.num_nodes()).topk(ct)
    protected = torch.zeros(self.graph.num_nodes(), dtype=torch.bool)
    protected[sample.indices] = True
    self.set_protected(protected)

  def get_sample_graph(self, edge_ct: int, root_graph: dgl.DGLGraph) -> dgl.DGLGraph:
    adj = root_graph.adj().int().to_dense()
    cutoff = 1 - (edge_ct / adj.shape[0] ** 2) 
    sample = (torch.rand(adj.shape) > cutoff).int()

    positive_samples = (sample * adj).to_sparse().float()
    negative_samples = (sample * (adj - 1)).to_sparse().float()
    negative_samples.values()[:] = 0
    all_samples = positive_samples + negative_samples
    idx = all_samples.indices()

    sample_graph = dgl.graph((idx[0], idx[1]), num_nodes=root_graph.num_nodes())

    # 1 is positive sample, 0 is negative sample
    sample_graph.edata['sample_type'] = all_samples.values()
    sample_graph.edata['sample_type'].requires_grad_(True)

    return sample_graph

  def create_locked_graph(self, attack_epochs: int, surrogate: torch.nn.Module) -> dgl.data.DGLDataset:
    assert 'protected' in self.dataset.ndata

    t = tqdm(range(attack_epochs), bar_format='{l_bar}{bar:10}{r_bar}{bar:-10b}')
    t.set_description("Perturbing")

    for epoch in t:
      None


attack = LearnabilityLock(dataset)
attack.designate_protected_random(node_ct=100)
attack.get_sample_graph(1000, root_graph=attack.graph)


# attack.create_locked_graph(attack_epochs=10, surrogate=surrogate)

Graph(num_nodes=2708, num_edges=1033,
      ndata_schemes={}
      edata_schemes={'sample_type': Scheme(shape=(), dtype=torch.float32)})

In [81]:
modified_adj.edges()

(tensor([   3,    5,    6,  ..., 2704, 2707, 2706]),
 tensor([   0,    0,    0,  ..., 2705, 2706, 2707]))

In [6]:
graph = dataset
ct = 100

adj = graph.adj().int().to_dense()
sample = (torch.rand(adj.shape) > 0.9).int()

positive_samples = (sample * adj).to_sparse().float()
negative_samples = (sample * (adj - 1)).to_sparse().float()
negative_samples.values()[:] = 0
all_samples = positive_samples + negative_samples
idx = all_samples.indices()

sample_graph = dgl.graph((idx[0], idx[1]), num_nodes=graph.num_nodes())

# 1 is positive sample, 0 is negative sample
sample_graph.edata['sample_type'] = all_samples.values()
sample_graph.edata['sample_type'].requires_grad_(True)

tensor([0., 0., 0.,  ..., 0., 0., 0.], requires_grad=True)

In [63]:
import torch.nn.functional as F

g0 = (torch.rand(dataset.num_nodes()) > 0.8) == 1
gX = ~g0

for epoch in range(2):
    accum_grad = torch.zeros_like(sample_graph.adj().to_dense())
    # modified_adj = modified_adj.add_self_loop()
    pred = surrogate.forward_positive(sample_graph, dataset.ndata['feat'], sample_graph.edata['sampleType'])

    loss = F.cross_entropy(pred[g0], dataset.ndata['label'][g0]) \
        - F.cross_entropy(pred[gX], dataset.ndata['label'][gX])

    loss.backward()
    sample_graph.edata['sample_type'].grad
    sampled_edges = sample_graph.edges()
    accum_grad[sampled_edges] += sample_graph.edata['sample_type'].grad
# modified_adj.adj()

In [149]:
a = torch.tensor([[1, 1, 0], [0, 1, 1], [0, 0, 0]]).to_sparse()
b = torch.tensor([[0, 1, 0], [0, 0, 1], [0, 0, 0]]).to_sparse()

a + b

tensor(indices=tensor([[0, 0, 1, 1],
                       [0, 1, 1, 2]]),
       values=tensor([1, 2, 1, 2]),
       size=(3, 3), nnz=4, layout=torch.sparse_coo)

In [204]:
def s(t: torch.Tensor):
  print(f'shape:{t.shape}\tsum:{t.sum()}\tmin:{t.min()}\tmax: {t.max()}\tmean:{t.mean()}')

In [205]:
coalesced = accum_grad.coalesce()
averaged = coalesced.values() / counts.coalesce().values()
averaged[averaged != averaged] = 0
adj_grad = torch.sparse_coo_tensor(coalesced.indices(), averaged)
s(adj_grad.coalesce().values())

shape:torch.Size([486708])	sum:-2.199384653067682e-06	min:-5.335767809810932e-07	max: 4.758028069318243e-07	mean:-4.51889957744922e-12


In [332]:
import numpy as np

g0 = attack.graph.ndata['protected']
gX = ~g0
num_perturbations = 1000
modified_graph = attack.graph.clone()

for epoch in range(10):
  accum_grad = torch.zeros_like(attack.graph.adj())
  counts = torch.zeros_like(accum_grad)
  perturbations = torch.zeros_like(accum_grad)

  # Sampling to create adj_grad
  for k in range(5):
    sample_graph = attack.get_sample_graph(edge_ct=100000, root_graph=modified_graph)

    pred = surrogate.forward_positive(
      g=sample_graph, 
      feat=attack.graph.ndata['feat'], 
      edge_weight=sample_graph.edata['sample_type'])
    
    # Composite loss
    loss = F.cross_entropy(pred[g0], attack.graph.ndata['label'][g0]) \
      - F.cross_entropy(pred[gX], attack.graph.ndata['label'][gX])
    loss.backward()

    # Gradient clipping
    clip_grad = sample_graph.edata['sample_type'].grad
    # positive_edges = sample_graph.edata['sample_type'] == 1
    # # Clip negative node gradients to only positive values
    # F.relu_(clip_grad[~positive_edges]) 
    # # Clip positive node gradients to only negative values
    # clip_grad[positive_edges] = -F.relu(-clip_grad[positive_edges])
    grad_graph = torch.sparse_coo_tensor(torch.stack(sample_graph.edges()), clip_grad)

    # accum_grad[sample_graph.edges()] += clip_grad # Accumulate gradient
    accum_grad = accum_grad + grad_graph
    counts = counts + grad_graph.bool()
    # counts[sample_graph.edges()] += 1 # Count for average

  accum_grad = accum_grad.coalesce()
  averaged = accum_grad.values() / counts.coalesce().values() # Average
  averaged[averaged != averaged] = 0 # Remove NaN
  accum_grad.values()[:] = averaged
  # adj_grad = torch.sparse_coo_tensor(accum_grad.indices(), averaged)
  # adj_grad = adj_grad.triu(diagonal=1) # Keep only valid adj, without diagonal

  lr = 1000000 / accum_grad.values().sum()
  perturbations = perturbations + (lr * accum_grad).coalesce() # Accumulate perturbations

  print((lr * accum_grad).coalesce().values().sum())
  # A simple projection to scale up the gradient
  for i in range(10):
    perturbations = perturbations * (num_perturbations / perturbations.values().abs().sum())
    perturbations.values().clamp_(-1, 1)
    if np.abs(1 - (perturbations.values().abs().sum() / num_perturbations)) < 0.1:
      break

  # Get the modified adj from perturbations
  modified_adj = attack.graph.adj().to_dense()
  idx = torch.bernoulli(perturbations.values().abs()) == 1
  adj_target_idx = perturbations.indices()[:, idx]
  modified_adj[adj_target_idx[0], adj_target_idx[1]] = \
    1 - modified_adj[adj_target_idx[0], adj_target_idx[1]]

  # Update the locked graph
  edges = modified_adj.to_sparse().indices()
  modified_graph = dgl.graph((edges[0], edges[1]))

  # Train the model
  surrogate.fit(
    g=modified_graph, 
    feat=attack.graph.ndata['feat'], 
    labels=attack.graph.ndata['label'], 
    epochs=1, 
    verbose=True)

tensor(1000000.)


GCN Training: 100%|██████████| 1/1 [00:00<00:00, 85.97it/s, loss=0.79]


tensor(999999.6250)


GCN Training: 100%|██████████| 1/1 [00:00<00:00, 111.71it/s, loss=0.76]


tensor(1000000.)


...

GCN Training: 100%|██████████| 1/1 [00:00<00:00, 101.54it/s, loss=0.77]


tensor(1000000.1250)


GCN Training: 100%|██████████| 1/1 [00:00<00:00, 99.71it/s, loss=0.76]


tensor(1000000.1875)


GCN Training: 100%|██████████| 1/1 [00:00<00:00, 105.88it/s, loss=0.76]


In [333]:
# Evaluate
import sklearn.metrics as metrics

locked_graph = modified_graph

lock_model = models.GCN(
  in_size=dataset.ndata['feat'].shape[1],
  out_size=int(dataset.ndata['label'].max()) + 1,
  hid_size=config['hid_size'],
  lr=config['lr'],
  dropout=config['dropout'],
  weight_decay=config['weight_decay']
)

lock_model.fit(
    g=locked_graph, 
    feat=attack.graph.ndata['feat'], 
    labels=attack.graph.ndata['label'], 
    epochs=200, 
    verbose=True)

pred = lock_model(dataset, dataset.ndata['feat'])
f1_g0 = metrics.f1_score(dataset.ndata['label'][g0], pred.argmax(dim=1)[g0], average='micro')
f1_gX = metrics.f1_score(dataset.ndata['label'][gX], pred.argmax(dim=1)[gX], average='micro')
f1_g0, f1_gX

# match = pred.argmax(dim=1) == dataset.ndata['label']
# match.sum() / match.shape[0]

GCN Training: 100%|██████████| 200/200 [00:01<00:00, 134.23it/s, loss=0.66]


(0.81, 0.8546779141104295)